In [11]:
from keras import backend as K
import tensorflow as tf
from math import ceil
import numpy as np
import math
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy32
from bounding_box_utils.bounding_box_utils import iou_float, convert_coordinates
import timeit

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
K.set_session(sess)
K.clear_session()

def eva(x):
    return(K.eval(x))

In [13]:
def tf_intersection_area_(boxes1, boxes2, coords='corners', mode='outer_product', border_pixels='half'):
    m = tf.shape(boxes1)[0]
    n = tf.shape(boxes2)[0]
    
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3

    d = 0
    
    box1xminxmax = tf.slice(boxes1, [0, 0], [m, xmax])
    box2xminxmax = tf.slice(boxes2, [0, 0], [n, xmax])                      
                            
    min_xy = tf.maximum(tf.tile(tf.expand_dims(box1xminxmax,1),[1, n, 1]),
                        tf.tile(tf.expand_dims(box2xminxmax,0),[m, 1, 1]))
    
    box1xmaxymax = tf.slice(boxes1, [0, 2], [m, 2])
    box2xmaxymax = tf.slice(boxes2, [0, 2], [n, 2])                      

    max_xy = tf.minimum(tf.tile(tf.expand_dims(box1xmaxymax,1),[1, n, 1]),
                    tf.tile(tf.expand_dims(box2xmaxymax,0),[m, 1, 1]))

    side_lengths = tf.maximum(tf.cast(0, dtype=tf.int64), tf.add(tf.subtract(max_xy,min_xy),d))

    return tf.multiply(side_lengths[:,:,0],side_lengths[:,:,1])

In [14]:
def tf_iou(boxes1, boxes2, coords='centroids', mode='outer_product', border_pixels='half'):

    intersection_areas = tf_intersection_area_(boxes1, boxes2, coords=coords, mode=mode)
    m = tf.shape(boxes1)[0]
    n = tf.shape(boxes2)[0]
    
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3

    d = 0
    
    boxes1_areas = tf.tile(tf.expand_dims(tf.multiply(tf.add(tf.subtract(boxes1[:,xmax],boxes1[:,xmin]),d),tf.add(tf.subtract(boxes1[:,ymax],boxes1[:,ymin]),d)), 1), [1,n])
    boxes2_areas = tf.tile(tf.expand_dims(tf.multiply(tf.add(tf.subtract(boxes2[:,xmax],boxes2[:,xmin]),d),tf.add(tf.subtract(boxes2[:,ymax],boxes2[:,ymin]),d)), 0), [m,1])
    
    union_areas = tf.subtract(tf.add(boxes1_areas,boxes2_areas),intersection_areas)
    op = tf.divide(intersection_areas,union_areas)
    return op

In [30]:
y = np.load('y_pred.npy')
x = np.load('y_true.npy')

y = tf.convert_to_tensor(y)
x = tf.convert_to_tensor(x)

iou_out = tf_iou(x,y)

In [41]:
K.eval(t)

array([[-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -0., -0., -0.],
       [ 0., -0.,  0., ..., -0., -0.,  0.],
       [-0., -0., -0., ...,  0.,  0.,  0.]], dtype=float32)

In [59]:
t = tf.cast(t, dtype=tf.float32)
rows = tf.cast(50, dtype=tf.float32)
iyter,rr  = tf.contrib.image.bipartite_match(t,rows,10)

In [60]:
print("iyter: ",K.eval(iyter))
print("rr: ",K.eval(rr))

iyter:  [ 0 12 26 -1 -1  4 -1 -1 -1 -1 10 -1 -1 -1 -1 -1 -1 -1 -1 -1 22 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 46 -1 -1 -1 -1 -1 -1 47
 48 49]
rr:  [ 0 -1 -1 -1  5 -1 -1 -1 -1 -1 10 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 20 -1
 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 40 47
 48 49]


In [39]:
def _match(similarity_matrix, valid_rows):
    valid_row_sim_matrix = tf.gather(similarity_matrix,
                                     tf.squeeze(tf.where(valid_rows), axis=-1))
    invalid_row_sim_matrix = tf.gather(
        similarity_matrix,
        tf.squeeze(tf.where(tf.logical_not(valid_rows)), axis=-1))
    similarity_matrix = tf.concat(
        [valid_row_sim_matrix, invalid_row_sim_matrix], axis=0)

    distance_matrix = -1 * similarity_matrix
    num_valid_rows = tf.reduce_sum(tf.to_float(valid_rows))
    _, match_results = image_ops.bipartite_match(
        distance_matrix, num_valid_rows=num_valid_rows)
    match_results = tf.reshape(match_results, [-1])
    match_results = tf.cast(match_results, tf.int32)
    return match_results


In [40]:
outw = _match(t,rows)
    

ValueError: Can not squeeze dim[1], expected a dimension of 1, got 0 for 'Squeeze' (op: 'Squeeze') with input shapes: [?,0].

In [ ]:
match_bipartite_greedy_tf(t)